In [1]:
import os
os.environ['WANDB_DISABLED'] = 'true'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from datasets import load_dataset
from tqdm.auto import tqdm
import warnings; warnings.filterwarnings('ignore')
from transformers import BertModel, BertTokenizer
import torch
import seaborn as sns
import matplotlib.pyplot as plt

/home/bowen/miniconda3/envs/552/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install contractions

# Data + preprocessing

In [3]:
data = pd.read_csv("/home/bowen/singlecell/ECSE552/final_project/2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.1.csv")
data.head()

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
3,4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm


In [4]:
data.drop(columns = ['Unnamed: 0'], axis = 1 , inplace = True)
data.head()

,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
1,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
2,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
3,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
4,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40623 entries, 0 to 40622
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              40623 non-null  object 
 1   text            40623 non-null  object 
 2   label           40623 non-null  object 
 3   type            40623 non-null  object 
 4   model_wrong     26097 non-null  object 
 5   db.model_preds  26097 non-null  float64
 6   status          40623 non-null  object 
 7   round           40623 non-null  object 
 8   split           40623 non-null  object 
 9   annotator       40623 non-null  object 
dtypes: float64(1), object(9)
memory usage: 3.1+ MB


In [6]:
data['id'].dtype

dtype('O')

In [7]:
## drop out NaN values
for i in data.columns:
    print(i,":",data[str(i)].isnull().sum()/data.shape[0])
data.dropna(axis = 0,inplace = True)
print("after drop out NaN values")
for i in data.columns:
    print(i,":",data[str(i)].isnull().sum()/data.shape[0])

id : 0.0
text : 0.0
label : 0.0
type : 0.0
model_wrong : 0.35758068089505946
db.model_preds : 0.35758068089505946
status : 0.0
round : 0.0
split : 0.0
annotator : 0.0
after drop out NaN values
id : 0.0
text : 0.0
label : 0.0
type : 0.0
model_wrong : 0.0
db.model_preds : 0.0
status : 0.0
round : 0.0
split : 0.0
annotator : 0.0


In [8]:
train_df = data[data['split'] == 'train'] # split the dataset by the column 'split' which labels 'train' and 'test' samples
test_df = data[data['split'] == 'test'] # split the dataset by the column 'split' which labels 'train' and 'test' samples
     

In [9]:
train_df = train_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0
test_df = test_df.replace({'label': {'hate': 1, 'nothate': 0}}) # relabel the `label` column, hate is 1 and nothate is 0

In [10]:
train_df = train_df.rename(columns={'label': 'labels'})
test_df = test_df.rename(columns={'label': 'labels'})
data = [[train_df.labels.value_counts()[0], test_df.labels.value_counts()[0]], 
        [train_df.labels.value_counts()[1], test_df.labels.value_counts()[1]]]
# Prints out the dataset sizes of train test and validate as per the table.
pd.DataFrame(data, columns=['Train', 'Test'])

,Train,Test
0,8961,861
1,12588,1234


In [11]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 128,
    'fp16': True,
    'output_dir': '/home/bowen/singlecell/ECSE552/final_project/outputs/'
}

In [15]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args,use_cuda=False) 

# Train the model, there is no development or validation set for this dataset 
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.json HTTP/1.1" 200 0


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled
100%|██████████| 21549/21549 [00:02<00:00, 10418.88it/s]
INFO:simpletransformers.classification.classification_model: 23166 features created from 21549 samples.
Epoch 1 of 1: 100%|██████████| 1/1 [11:04<00:00, 664.64s/it]
INFO:simpletransformers.classification.classification_model: Training of roberta model complet